In [ ]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.2 MB/s eta 0:00:00


In [ ]:
import plotly.io as pio

In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime, timedelta
import time
import plotly.graph_objects as go

In [ ]:
today = date.today()
today_date = datetime.today().date()
first_reg = datetime(1997, 1, 1).date()

In [ ]:
diff = (today_date - first_reg).days

In [ ]:
dates_list = pd.date_range(first_reg, today_date)

In [ ]:
ranges = [(i, min(i + 3000, diff)) for i in range(0, diff, 3000)]

In [ ]:
dbs = []
for range in ranges:
  start_date = str(dates_list[range[0]].date())
  last_date = str(dates_list[range[1]].date())
  url = f"https://api.bcra.gob.ar/estadisticas/v3.0/monetarias/1?desde={start_date}&hasta={last_date}&limit=3000"
  db = requests.get(url, verify=False).json()["results"]
  db = pd.DataFrame(db)[["fecha", "valor"]]
  db.rename(columns = {"valor": "RRII"}, inplace=True)
  db["fecha"] = pd.to_datetime(db["fecha"])
  db = db.sort_values('fecha').reset_index(drop=True)
  db = db[~db["fecha"].dt.dayofweek.isin([5, 6])].reset_index(drop=True)
  db["year"] = db["fecha"].dt.year
  dbs.append(db)
df = pd.concat(dbs, ignore_index=True)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bcra.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bcra.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.bcra.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1

In [ ]:
df["v_abs"] = df["RRII"] - df["RRII"].shift(1)

In [ ]:
df["v_rel"] = (df["RRII"]/df["RRII"].shift(1) - 1)*100

In [ ]:
def create_heatmap(df, variable, zmin, zmax, fig):
  for i, year in enumerate(pd.unique(df["year"])):
    df_year = df[df["year"] == year].groupby(["fecha"], as_index=False)[[variable]].sum().sort_values(by="fecha")
    df_long = pd.DataFrame()
    df_long["fecha"] = pd.date_range(datetime(year,1,1), datetime(year,12,31))
    df_year = df_long.merge(df_year, on ="fecha", how="outer")
    df_year["date"] = df_year["fecha"].apply(lambda x: x.replace(year=1904))

    fig.add_trace(
        go.Heatmap(
        x = [year]*len(df_year),
        y = df_year["date"].tolist(),
        z = df_year[variable].tolist(),
        showscale=(i == 0),
        colorscale = "RdBu",
        zmin = zmin,
        zmid = 0,
        zmax = zmax,
        hovertemplate='%{y}/%{x}<br>Var.: %{z}<extra></extra>',
        colorbar=dict(
                  orientation="h",
                  x=0.5,
                  y=1.05,
                  xanchor="center",
                  yanchor="top",
                  len=0.5
              ) if i == 0 else None
          )
    )

In [ ]:
def update_layout(fig, abs_rel):
  fig.update_layout(
    plot_bgcolor ='rgba(0,0,0,0)',
    margin = dict(
        t = 200,
        b = 50,
        r = 25,
        l = 75
    ),
    height=1500,
    width=1000,
    font = dict(
        family="Courier New"
    ),
    title = dict(
      text = f"<b>Foreign exchange reserves</b><br><sup>{abs_rel} day over day changes in the gross stock of Foreign Exchange Reserves.<br><sup><b>Source:</b> BCRA</sup></sup>",
      xanchor = 'left',
      x = 0.0325,
      yanchor = 'top',
      y = 0.95,
      font = dict(
          size = 24,
          color = 'black'
      )
    ),
    yaxis=dict(
          autorange="reversed",
          title = None,
          tick0="1904-01-01",
          dtick="M1",
          tickformat="%b - %d",
          tickangle = 0
        ),
    xaxis=dict(
        dtick=1
    )
  )

In [ ]:
absolute_fig = go.Figure()

create_heatmap(df, "v_abs", -1000, 1000, absolute_fig)
update_layout(absolute_fig, "Absolute")

In [ ]:
absolute_fig.show()

In [ ]:
pio.write_html(absolute_fig,
               file='fig_var_RRII.html',
               full_html=True)

In [ ]:
relative_fig = go.Figure()

create_heatmap(df, "v_rel", -5, 5, relative_fig)
update_layout(relative_fig, "Relative")

In [ ]:
relative_fig.show()